In [1]:
# prompting a custom chat model
from scai.custom_chat_models.crfm import crfmChatLLM

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd" # gpt4
# CRFM_API_KEY = "FP9qVrZnCjCmxsrsCemxhEzrDUr9Oo1q" # 003

chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
                   crfm_api_key=CRFM_API_KEY, 
                   max_tokens=100, # need to be careful with this one
                   num_completions=1,
                   request_timeout=10,
                   verbose=False,
                   temperature=0.9,
                   )

In [2]:
# method 2: using prompt templates (this is what we will do for LLM chains? returns strings!)
from langchain import PromptTemplate, LLMChain

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


template="You are a helpful assistant."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
ai_template="{response}"
ai_message_prompt_1 = AIMessagePromptTemplate.from_template(ai_template)
human_message_prompt_1 = HumanMessagePromptTemplate.from_template(human_template)
human_message_prompt_2 = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt_1] )

chain = LLMChain(llm=chat_llm, prompt=chat_prompt)
response = chain.run(input="cats", text="tell me about a funy cat.", stop=["System:"])
print(response)

Once upon a time, there was a hilarious cat named Whiskers. Whiskers was known by everyone in the neighborhood, not just because of his fluffy, vibrant orange fur, but also because of his knack for getting into comical situations. Neighbors would often find Whiskers trying to squeeze his plump body into tiny boxes, swinging from tree branches by his tail, or even trying to hitch a ride on the unsuspecting mail carrier's vehicle.

Whiskers was always looking for


In [3]:
# method 1: using messages directly
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You are a helpful AI Assistant"),
    HumanMessage(content="Tell me a joke"),
    AIMessage(content="What do you call a cat that can do magic tricks? A magic kit."),
    HumanMessage(content="This joke was not funny.") ,
]

response = chat_llm.generate([messages], stop=["System:"])

for i, generation in enumerate(response.generations[0]):
    print(f"AI Message {i}: {generation.text}")

AI Message 0: I'm sorry if you didn't find that one funny. Let me try another one for you. Why don't scientists trust atoms? Because they make up everything!


In [20]:
# method 2: using prompt templates (this is what we will do for LLM chains? returns strings!)
from langchain import PromptTemplate, LLMChain

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


template="You are a helpful assistant."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template_0 ="Tell me a joke, a funny one. font talk about scientists"
human_message_prompt_0 = HumanMessagePromptTemplate.from_template(human_template_0)
assistant_template_0 = "What do you call a cat that can do magic tricks? A magic kit."
ai_message_prompt_0 = AIMessagePromptTemplate.from_template(assistant_template_0)
human_template_1 ="The joke wasnt funny. Tell me another one and also tell me how it is better from the one above. Repeat the one from above as well."
human_message_prompt_1 = HumanMessagePromptTemplate.from_template(human_template_1)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt_0])

chain = LLMChain(llm=chat_llm, prompt=chat_prompt)
response = chain.run(input_language="English", output_language="French", text="I love programming.", stop=["System:"])
print(response)

Why don't some couples go to the gym? Because some relationships don't work out!


In [29]:
# using a free hugging face model 
import os
from langchain import HuggingFaceHub

HUGGINGFACEHUB_API_TOKEN = "YOUR_HF_API_TOKEN"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

repo_id = "gpt2"

llm = HuggingFaceHub(repo_id=repo_id, 
                     model_kwargs={"temperature":0.8, "max_length":100,"num_return_sequences":2})




template="What comes after 1?"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="2. And the next number is:"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=llm, prompt=chat_prompt)
response = chain.run(stop=["System:"])
print(response)

ValueError: Error raised by inference API: Authorization header is correct, but the token seems invalid

In [10]:
# checking how much tokens we have left (@TODO JP: need to make sure we get these for a given simulation)
from helm.proxy.services.remote_service import RemoteService
from helm.common.authentication import Authentication
from helm.proxy.accounts import Account

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd" # gpt4 # from philipp
CRFM_API_KEY = "PncXrdFlPLSopZUeu6eqqfSwq9DKte1m" # davinci-003 # from kanishk
# add your keys here! 

# An example of how to use the request API.
auth = Authentication(api_key=CRFM_API_KEY)
service = RemoteService("https://crfm-models.stanford.edu")

# Access account and show my current quotas and usages (please monitor)
model = "gpt4" 
model = "gpt3"
account: Account = service.get_account(auth)
for i, k in enumerate(account.usages[model]):
    print(f"Model: {model}, Usage {i}: {k}: {account.usages[model][k]}")

Model: gpt3, Usage 0: daily: Usage(period='2023-6-12', used=40675, quota=None)
Model: gpt3, Usage 1: monthly: Usage(period='2023-6', used=305376, quota=None)
Model: gpt3, Usage 2: total: Usage(period='all', used=9740132, quota=12000000)
